In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uci-drug/UCIdrug_train.csv
/kaggle/input/uci-drug/UCIdrug_test.csv


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('/kaggle/input/uci-drug/UCIdrug_train.csv')
test = pd.read_csv('/kaggle/input/uci-drug/UCIdrug_test.csv')
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [4]:
# as both the dataset contains same columns we can combine them for better analysis

data = pd.concat([df, test])
data.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [5]:
# describing the data

data.describe()

,uniqueID,rating,usefulCount
count,215063.000000,215063.000000,215063.000000
mean,116039.364814,6.990008,28.001004
std,67007.913366,3.275554,36.346069
min,0.000000,1.000000,0.000000
25%,58115.500000,5.000000,6.000000
50%,115867.000000,8.000000,16.000000
75%,173963.500000,10.000000,36.000000
max,232291.000000,10.000000,1291.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215063 entries, 0 to 53765
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     215063 non-null  int64 
 1   drugName     215063 non-null  object
 2   condition    213869 non-null  object
 3   review       215063 non-null  object
 4   rating       215063 non-null  int64 
 5   date         215063 non-null  object
 6   usefulCount  215063 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 13.1+ MB


In [7]:
data.dtypes

uniqueID        int64
drugName       object
condition      object
review         object
rating          int64
date           object
usefulCount     int64
dtype: object

In [8]:
data.isnull().any()

uniqueID       False
drugName       False
condition       True
review         False
rating         False
date           False
usefulCount    False
dtype: bool

In [9]:
# Dropping the data points with null values 
data = data.dropna(how = 'any', axis = 0)
# lowercasing the column names so it will be easier for access ^^
data.columns = data.columns.str.lower()
# Sorting the dataframe
data.sort_values(['rating'], ascending = True, inplace = True)
data.reset_index(drop = True, inplace = True)
# Converting the date in to date time format 
data['date'] = pd.to_datetime(data['date'])
data.head(10)

/tmp/ipykernel_18/3170060005.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date'] = pd.to_datetime(data['date'])


,uniqueid,drugname,condition,review,rating,date,usefulcount
0,213924,Tioconazole,Vaginal Yeast Infection,"""OMG! Only the 2nd YI I have ever had &amp; th...",1,2016-09-24,12
1,90417,Docosanol,Herpes Simplex,"""I woke up in the morning with redness at the ...",1,2017-02-16,19
2,208407,Oseltamivir,Influenza,"""Tamil the first day seemed to just make me na...",1,2015-12-21,10
3,101350,Aubra,Birth Control,"""I went on Aubra to 1. Avoid any accidents wit...",1,2016-08-19,0
4,56915,Ranitidine,Stomach Ulce,"""Didn&#039;t take away pain in 1 months""",1,2016-01-14,4
5,144594,Etonogestrel,Birth Control,"""Awful awful awful!!! Lost more than half of m...",1,2016-03-13,2
6,16136,Ethinyl estradiol / norethindrone,Menstrual Disorders,"""I&#039;m 41. My latest check up showed that I...",1,2016-05-02,2
7,154778,Metronidazole,Bacterial Infection,"""After a tooth infection, I was prescribed the...",1,2017-09-08,1
8,64313,Sertraline,Depression,"""This is about my woman who is on zoloft.........",1,2016-09-21,13
9,53559,Alesse,Birth Control,"""I hated this bcp. I gained 15 pounds in two w...",1,2017-01-03,4


In [10]:
# Total unique conditions in the dataset
print (data['condition'].nunique(), "\n")

print ("some of the conditions are : ", data['condition'].unique()[0:10])

916 

some of the conditions are :  ['Vaginal Yeast Infection' 'Herpes Simplex' 'Influenza' 'Birth Control'
 'Stomach Ulce' 'Menstrual Disorders' 'Bacterial Infection' 'Depression'
 'Migraine Prevention' 'ADHD']


In [11]:

# Giving the Sentiment according to the ratings
import math
data['sentiment_rate'] = data['rating'].apply(lambda x: math.ceil(x/2))
data['sentiment_rate'].value_counts()

sentiment_rate
5    104181
1     37972
4     37379
3     19053
2     15284
Name: count, dtype: int64

In [12]:
# #applying the above assumptions into a column
# data['review_clean'] = review_clean(data['review'])

# # Expanding the contractions
# data['review_clean'] = data['review_clean'].apply(lambda x: expand_contractions(x))

# # Removing punctuations
# data['review_clean'] = data['review_clean'].apply(lambda x: ''.join(word for word in x if word n

In [13]:
from textblob import TextBlob
"""I have used textblob module to give the sentiment polarity of the review. This polarity is given to both the cleaned and uncleaned review"""
#from nltk.corpus import stopwords
from collections import Counter
import warnings; warnings.simplefilter('ignore')
import nltk
import string
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

# Removing the stopwords from the review_clean column
stop_words = set(stopwords.words('english'))
data['review_clean'] = data['review_clean'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

# Removing the word stems using the Snowball Stemmer
Snow_ball = SnowballStemmer("english")
data['review_clean'] = data['review_clean'].apply(lambda x: " ".join(Snow_ball.stem(word) for word in x.split()))

# Separating the day, month and year from the Date

data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year

data.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'stopwords' is not defined

In [ ]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS

stopwords = set(STOPWORDS)

wordcloud = WordCloud(background_color = 'orange', stopwords = stopwords, width = 1200, height = 800).generate(str(data['drugName']))

plt.rcParams['figure.figsize'] = (15, 15)
plt.title('Word Cloud - Drug Names', fontsize = 25)
print(wordcloud)
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

In [ ]:
def review_clean(review):
    # changing to lower case
    lower = review.str.lower()

    # Replacing the repeating pattern of &#039;
    pattern_remove = lower.str.replace("&#039;", "")

    # Removing all the special Characters
    special_remove = pattern_remove.str.replace(r'[^\w\d\s]',' ')

    # Removing all the non ASCII characters
    ascii_remove = special_remove.str.replace(r'[^\x00-\x7F]+',' ')

    # Removing the leading and trailing Whitespaces
    whitespace_remove = ascii_remove.str.replace(r'^\s+|\s+?$','')

    # Replacing multiple Spaces with Single Space
    multiw_remove = whitespace_remove.str.replace(r'\s+',' ')

    # Replacing Two or more dots with one
    dataframe = multiw_remove.str.replace(r'\.{2,}', ' ')

    return dataframe

In [ ]:
data['review_clean'] = review_clean(data['review'])

In [ ]:
data['review_clean']

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from textblob import TextBlob
from nltk.corpus import stopwords
from collections import Counter
import warnings; warnings.simplefilter('ignore')
import nltk
import string
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Removing the stopwords
stop_words = set(stopwords.words('english'))
data['review_clean'] = data['review_clean'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [ ]:
# # Removing the word stems using the Snowball Stemmer
# Snow_ball = SnowballStemmer("english")
# data['review_clean'] = data['review_clean'].apply(lambda x: " ".join(Snow_ball.stem(word) for word in x.split()))

In [ ]:
# from nltk.stem import SnowballStemmer
# import pandas as pd

# # Create a SnowballStemmer object
# Snow_ball = SnowballStemmer("english")

# # Define a function to stem words in a text
# def stem_words(text):
#     stemmed_words = [Snow_ball.stem(word) for word in text.split()]
#     return " ".join(stemmed_words)

# # Apply the stemming function to the 'review_clean' column
# data['review_clean'] = data['review_clean'].apply(stem_words)


In [ ]:
data.columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd

# Assume your data is stored in a DataFrame called 'df'

# Drop rows with missing values in 'condition' column
data.dropna(subset=['condition'], inplace=True)

# Select relevant features
X = data[['drugName', 'condition', 'review', 'rating', 'usefulCount']]
y = data['rating']  # Target variable

# Encode categorical variables
X_encoded = pd.get_dummies(X, columns=['drugName', 'condition'])

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Normalize or standardize numerical features
scaler = StandardScaler()
X_train[['rating', 'usefulCount']] = scaler.fit_transform(X_train[['rating', 'usefulCount']])
X_test[['rating', 'usefulCount']] = scaler.transform(X_test[['rating', 'usefulCount']])


In [ ]:
# # Assuming your cleaned and organized text data is stored in a DataFrame called 'df'

# # Concatenate all reviews into a single document
# corpus = ' '.join(df['review'])

# # Alternatively, create a list of documents where each element represents a review
# corpus_list = df['review'].tolist()


In [ ]:
# print(corpus_list)

In [ ]:
# # Assuming your cleaned and organized text data is stored in a DataFrame called 'df'

# # Concatenate all reviews into a single document
# corpus = ' '.join(df['review'])

# # Alternatively, create a list of documents where each element represents a review
# corpus_list = df['review'].tolist()

# # Print the concatenated corpus
# print(corpus)


In [ ]:
# Assuming your DataFrame is named df and the column containing reviews is named 'review'
# Check if the term "corpus" is present in each review
corpus_occurrences = df['review'].str.contains('corpus', case=False)

# Filter the DataFrame to get rows where the term "corpus" appears in the review
corpus_reviews = df[corpus_occurrences]

# Print the reviews where the term "corpus" is present
print(corpus_reviews['review'])


In [ ]:
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords if you haven't already
nltk.download('stopwords')

# Get the set of stopwords
stop_words = set(stopwords.words('english'))

# Example of additional words to remove
additional_stop_words = {'etc'}

# Combine the sets of stopwords
stop_words = stop_words.union(additional_stop_words)

# Function to remove stopwords from text
def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to your DataFrame column
df['clean_review'] = df['review'].apply(remove_stopwords)

# Show the DataFrame with cleaned reviews
print(df['clean_review'])


In [ ]:
# import nltk
# from nltk.tokenize import word_tokenize

# # Download NLTK tokenizer data if you haven't already
# nltk.download('punkt')

# # Function to tokenize text
# def tokenize_text(text):
#     tokens = word_tokenize(text)
#     return tokens

# # Apply the tokenize_text function to your DataFrame column
# df['tokenized_review'] = df['clean_review'].apply(tokenize_text)

# # Show the DataFrame with tokenized reviews
# print(df['tokenized_review'])


In [ ]:
# Function to tokenize text using split()
def tokenize_text(text):
    tokens = text.split()
    return tokens

# Apply the tokenize_text function to your DataFrame column
df['tokenized_review'] = df['clean_review'].apply(tokenize_text)

# Show the DataFrame with tokenized reviews
print(df['tokenized_review'])
